##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Salvar e carregar modelos

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O progresso dos modelos pode ser salvo durante e após o treinamento. Portanto, um modelo pode ser retomado de onde parou e evitar tempos de treinamento longos. Ao salvar, você também pode compartilhar o modelo, e outras pessoas podem recriar o seu trabalho. Ao publicar modelos e técnicas e pesquisa, a maioria dos pesquisadores de aprendizado de máquina compartilham:

- O código para criar o modelo.
- Os pesos treinados, ou parâmetros, do modelo.

Compartilhar esses dados ajuda outras pessoas a entenderem como o modelo funciona e testá-lo com novos dados.

Atenção: os modelos do TensorFlow são códigos, e é importante ter cuidado com código não confiável. Confira mais detalhes em [Como usar o TensorFlow com segurança](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md).

### Opções

Existem diferentes maneiras de salvar modelos do TensorFlow, dependendo da API usada. Este guia usa [tf.keras](https://www.tensorflow.org/guide/keras), uma API de alto nível para criar e treinar modelos no TensorFlow. O novo formato de alto nível `.keras` usado neste tutorial é recomendado para salvar objetos do Keras, pois oferece um salvamento robusto e eficiente baseado em nomes, que geralmente é mais fácil de depurar do que formatos legados ou de baixo nível. Para workflows de salvamento e serialização mais avançados, especialmente quando envolverem objetos personalizados, confira o [guia Salvar e carregar modelos do Keras](https://www.tensorflow.org/guide/keras/save_and_serialize). Para outras estratégias, confira o [guia Como usar o formato SavedModel](../../guide/saved_model.ipynb).

## Configuração

### Instalações e importações

Instale e importe o TensorFlow, além das dependências:

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Obter um dataset de exemplo

Para demonstrar como salvar e carregar pesos, você usará o [dataset MNIST](http://yann.lecun.com/exdb/mnist/). Para acelerar as execuções, use os primeiros mil exemplos:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Definir um modelo

Comece criando um modelo sequencial simples:

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

## Salvar checkpoints durante o treinamento

Você pode usar um modelo treinado sem precisar treiná-lo novamente ou pode retomar o treinamento de onde parou caso o processo de treinamento tenha sido interrompido. O callback `tf.keras.callbacks.ModelCheckpoint` permite salvar continuamente o modelo tanto *durante* quanto *no término* do treinamento.

### Uso do callback de checkpoint

Crie um callback `tf.keras.callbacks.ModelCheckpoint` que salve os pesos somente durante o treinamento:

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

É criada uma coleção única de arquivos de checkpoint do TensorFlow, atualizados no final de cada época:

In [ ]:
os.listdir(checkpoint_dir)

Desde que os dois modelos tenham a mesma arquitetura, você pode compartilhar pesos entre eles. Portanto, ao restaurar um modelo somente a partir dos pesos, crie um modelo com a mesma arquitetura do modelo original e depois defina os pesos.

Agora, crie novamente um modelo novo, não treinado, e faça a avaliação com o conjunto de teste. Um modelo não treinado será treinado ao acaso (exatidão de cerca de 10%):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

Em seguida, carregue os pesos do checkpoint e refaça a avaliação:

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

### Opções do callback de checkpoint

O callback oferece várias opções para fornecer nomes exclusivos para os checkpoints e ajustar a frequência de criação de checkpoints.

Treine um novo modelo e depois salve checkpoints com nomes exclusivos após cada cinco épocas:

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

Agora, confira os checkpoints resultantes e escolha o último:

In [ ]:
os.listdir(checkpoint_dir)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Observação: o formato padrão do TensorFlow salva somente os cinco checkpoints mais recentes.

Para testar, redefina o modelo e carregue o último checkpoint:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## O que são esses arquivos?

O código acima armazena os pesos em uma coleção de arquivos com formato de [checkpoint](../../guide/checkpoint.ipynb) que contêm somente os pesos treinados em um formato binário. Os checkpoints contêm:

- Um ou mais fragmentos que contêm os pesos do seu modelo.
- Um arquivo de índice que indica quais pesos são armazenados em qual fragmento.

Se você estiver treinando um modelo em uma única máquina, terá um fragmento com o sufixo: `.data-00000-of-00001`

## Salvar os pesos manualmente

Para salvar os pesos manualmente, use `tf.keras.Model.save_weights`. Por padrão, `tf.keras` — e especificamente o método `Model.save_weights` — usa o formato de [Checkpoint](../../guide/checkpoint.ipynb) do TensorFlow com extensão `.ckpt`. Para salvar no formato HDF5 com extensão `.h5`, confira o guia [Salvar e carregar modelos](https://www.tensorflow.org/guide/keras/save_and_serialize).

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## Salvar o modelo inteiro

Faça uma chamada a `tf.keras.Model.save` para salvar a arquitetura, os pesos e a configuração de treinamento de um modelo em um único arquivo zip `model.keras`.

Um modelo inteiro pode ser salvo em três formatos de arquivo diferentes (o novo formato `.keras` e dois formatos legados: `SavedModel` e `HDF5`). Ao salvar um modelo como `path/to/model.keras`, ele é salvo no formato mais recente automaticamente.

**Observação:** para objetos do Keras, recomenda-se usar o novo formato de alto nível `.keras`, pois ele oferece tarefas de salvar e carregar com base em nomes com mais recursos, além de ser mais fácil de depurar. O formato de baixo nível SavedModel e o formato legado H5 continuam com suporte para códigos existentes.

Para mudar para o formato SavedModel, você pode:

- Passar `save_format='tf'` para `save()`.
- Passar um nome de arquivo sem extensão.

Para mudar para o formato H5, você pode:

- Passar `save_format='h5'` para `save()`.
- Passar um nome de arquivo que termine em `.h5`.

É muito útil salvar um modelo totalmente funcional: você pode carregá-lo no TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)), depois treiná-lo e executá-lo em navegadores web, ou convertê-lo para ser executado em dispositivos móveis usando o TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_), [HDF5](https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_)).

*Objetos personalizados (por exemplo, modelos ou camadas que sejam uma subclasse) exigem atenção especial ao salvar e carregar. Confira a seção **Como salvar objetos personalizados** abaixo.

### O novo formato de alto nível `.keras`

O novo formato de salvamento do Keras v3, indicado pela extensão `.keras`, é um formato mais simples e eficiente que implementa salvamento baseado em nomes, garantindo que aquilo que você carregue seja exatamente o que foi salvo pela perspectiva do Python. Dessa forma, a depuração fica muito mais fácil, e ele é o formato recomendado para o Keras.

A seção abaixo ilustra como salvar e restaurar o modelo no formato `.keras`.

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Recarregue um novo modelo do Keras pelo arquivo zip `.keras`:

In [ ]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Tente executar a avaliação e a previsão com o modelo carregado:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

### Formato SavedModel

O formato SavedModel é outra forma de serializar modelos. Os modelos salvos nesse formato podem ser restaurados usando `tf.keras.models.load_model` e são compatíveis com o TensorFlow Serving. O [Guia do SavedModel](../../guide/saved_model.ipynb) detalha como `serve/inspect` (servir/inspecionar) o SavedModel. A seção abaixo ilustra as etapas para salvar e restaurar o modelo.

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

O formato SavedModel é um diretório que contém um binário protobuf e um checkpoint do TensorFlow. Avalie o diretório do modelo salvo:

In [ ]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

Recarregue um novo modelo do Keras a partir do modelo salvo:

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

O modelo restaurado é compilado com os mesmos argumentos do original. Tente executar a avaliação e previsão com o modelo carregado:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

### Formato HDF5

O Keras oferece um formato de salvamento de alto nível básico e legado usando o padrão [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Agora, recrie o modelo a partir desse arquivo:

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Confira a exatidão:

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

O Keras salva modelos avaliando suas arquiteturas. Essa técnica salva tudo:

- Os valores de pesos.
- A arquitetura do modelo.
- A configuração de treinamento do modelo (o que você passa para o método `.compile()`).
- O otimizador e seu estado, se houver (isso permite que você reinicie o treinamento de onde parou).

O Keras não consegue salvar os otimizadores `v1.x` (de `tf.compat.v1.train`), pois eles não são compatíveis com checkpoints. Para otimizadores v1.x, você precisa recompilar o modelo após carregá-lo, perdendo o estado do otimizador.


### Como salvar objetos personalizados

Se você estiver usando o formato SavedModel, pode ignorar esta seção. A diferença principal entre os formatos de alto nível `.keras`/HDF5 e o formato de baixo nível SavedModel é que os formatos `.keras`/HDF5 usam configurações de objetos para salvar a arquitetura do modelo, enquanto o formato SavedModel salva o grafo da execução. Portanto, os SavedModels conseguem salvar objetos personalizados, como modelos e camadas personalizadas que são uma subclasse, sem exigir o código original. Porém, uma consequência é que a depuração de SavedModels de baixo nível pode ser mais difícil, e recomendamos usar o formato de alto nível `.keras` por ser nativo do Keras e baseado em nomes.

Para salvar objetos personalizados como `.keras` e HDF5, você precisa:

1. Definir um método `get_config` no seu objeto e, opcionalmente, um método de classe `from_config`.
    - `get_config(self)` retorna um dicionário de parâmetros serializável em JSON, necessário para recriar o objeto.
    - `from_config(cls, config)` usa a configuração retornada por `get_config` para criar um novo objeto. Por padrão, essa função usará a configuração como kwargs de inicialização (`return cls(**config)`).
2. Passar os objetos personalizados para o modelo de uma dentre três formas:
    - Registrar o objeto personalizado com o decorador `@tf.keras.utils.register_keras_serializable` **(recomendado)**.
    - Passar o objeto diretamente para o argumento `custom_objects` ao carregar o modelo. O argumento precisa ser um dicionário que mapeie o nome da classe de string para a classe Python. Por exemplo: `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`.
    - Usar um `tf.keras.utils.custom_object_scope` com o objeto incluído no argumento de dicionário `custom_objects` e colocar uma chamada a `tf.keras.models.load_model(path)` dentro do escopo.

Confira o tutorial [Como escrever camadas e modelos do zero](https://www.tensorflow.org/guide/keras/custom_layers_and_models) para ver exemplos de objetos personalizados e `get_config`.
